In [ ]:
import SimpleITK as sitk
import os
import matplotlib.pyplot as plt

# Function to load DICOM series from a folder and resize if necessary
def load_and_resize_dicom_series(folder_path, target_size):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(folder_path)
    reader.SetFileNames(dicom_names)
    series = reader.Execute()
    
    # Check if resizing is necessary
    if any(size < 4 for size in series.GetSize()):
        print("Resizing series to meet minimum size requirements.")
        original_spacing = series.GetSpacing()
        original_size = series.GetSize()
        new_spacing = [original_spacing[i] * original_size[i] / target_size[i] for i in range(3)]
        resampler = sitk.ResampleImageFilter()
        resampler.SetSize(target_size)
        resampler.SetOutputSpacing(new_spacing)
        resampler.SetOutputOrigin(series.GetOrigin())
        resampled_series = resampler.Execute(series)
        return resampled_series
    else:
        return series

# Function to load DICOM series from a folder and accumulate them into a single series
def accumulate_series(folder_path):
    reader = sitk.ImageSeriesReader()
    dicom_names = reader.GetGDCMSeriesFileNames(folder_path)
    reader.SetFileNames(dicom_names)
    series = reader.Execute()
    return series

# Paths to CT and PET folders
ct_folder_path = "C:/Users/Lenovo/Desktop/ct"
pet_folder_path = "C:/Users/Lenovo/Desktop/pt"

# Output folder path
output_folder = "C:/Users/Lenovo/Desktop/p"

# Define target size for resizing
target_size = [128, 128, 64]  # Adjust as needed

# Load and accumulate CT and PET series
ct_series = accumulate_series(ct_folder_path)
pet_series = accumulate_series(pet_folder_path)

# Perform registration
registration_method = sitk.ImageRegistrationMethod()

# Set up registration parameters
registration_method.SetMetricAsMattesMutualInformation()
registration_method.SetMetricSamplingStrategy(registration_method.RANDOM)
registration_method.SetMetricSamplingPercentage(0.01)
registration_method.SetInterpolator(sitk.sitkLinear)
registration_method.SetOptimizerAsGradientDescent(learningRate=0.1, numberOfIterations=800, convergenceMinimumValue=1e-6, convergenceWindowSize=10)
registration_method.SetOptimizerScalesFromPhysicalShift()

# Use multi-resolution registration
registration_method.SetShrinkFactorsPerLevel([4, 2, 1])
registration_method.SetSmoothingSigmasPerLevel([4, 2, 1])
registration_method.SmoothingSigmasAreSpecifiedInPhysicalUnitsOn()

# Initialize transform
initial_transform = sitk.CenteredTransformInitializer(ct_series, pet_series, sitk.Euler3DTransform())

# Set initial transform
registration_method.SetInitialTransform(initial_transform)

# Perform registration
final_transform = registration_method.Execute(ct_series, pet_series)

# Resample PET series using the final transform
resampled_pet_series = sitk.Resample(pet_series, ct_series, final_transform, sitk.sitkLinear, 0.0, pet_series.GetPixelID())

# Apply denoising filter
denoised_pet_series = sitk.CurvatureFlow(image1=resampled_pet_series, timeStep=0.0625, numberOfIterations=50)

# Convert pixel type to unsigned 16-bit integer
resampled_pet_series_uint16 = sitk.Cast(sitk.RescaleIntensity(denoised_pet_series), sitk.sitkUInt16)

# Save the resulting image in the output folder
output_path = os.path.join(output_folder, "registered_pet_series.dcm")
sitk.WriteImage(resampled_pet_series_uint16, output_path)

# Convert SimpleITK image to numpy array
resampled_pet_array = sitk.GetArrayFromImage(resampled_pet_series_uint16)

# Plot all slices of the registered PET series with larger size
num_slices = resampled_pet_array.shape[0]
fig, axes = plt.subplots(1, num_slices, figsize=(20, 20))  # Increase figsize for larger slices

for i in range(num_slices):
    axes[i].imshow(resampled_pet_array[i], cmap='jet')
    axes[i].axis('off')
    axes[i].set_title('Slice {}'.format(i+1))

plt.tight_layout()
plt.show()

print("Registration complete. Registered PET series saved to:", output_path)
